In [1]:
sc.master

'local[*]'

In [29]:
if sc.master[0:5] == "local":
    path = "file:///root/data/members_v3.csv"
else:
    path = "hdfs://root/data/members_v3.csv"

# Establishing Spark DataFrame from RDD

In [30]:
rawDataWithHeader = sc.textFile(path)
rawDataWithHeader.take(2)

['msno,city,bd,gender,registered_via,registration_init_time',
 'Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,,11,20110911']

In [11]:
header = rawDataWithHeader.first()
rawData = rawDataWithHeader.filter(lambda x:x!=header)
rawField= rawData.map(lambda line:line.split(","))
print(header)
rawField.take(2)

msno,city,bd,gender,registered_via,registration_init_time


[['Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=',
  '1',
  '0',
  '',
  '11',
  '20110911'],
 ['+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=',
  '1',
  '0',
  '',
  '7',
  '20110914']]

In [35]:
from pyspark.sql import Row
member_rdd = rawField.map(lambda p:Row(msno=p[0],
                                          city=int(p[1]),
                                          bd=int(p[2]),
                                          gender=p[3],
                                          registered_via=int(p[4]),
                                          registration_init_time=(p[5])
                            ))
print(member_rdd.take(1))
member_df = sqlContext.createDataFrame(member_rdd)
print(member_df.printSchema())
print(member_df.show(5))

[Row(bd=0, city=1, gender='', msno='Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=', registered_via=11, registration_init_time='20110911')]
root
 |-- bd: long (nullable = true)
 |-- city: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- msno: string (nullable = true)
 |-- registered_via: long (nullable = true)
 |-- registration_init_time: string (nullable = true)

None
+---+----+------+--------------------+--------------+----------------------+
| bd|city|gender|                msno|registered_via|registration_init_time|
+---+----+------+--------------------+--------------+----------------------+
|  0|   1|      |Rb9UwLQTrxzBVwCB6...|            11|              20110911|
|  0|   1|      |+tJonkh+O1CA796Fm...|             7|              20110914|
|  0|   1|      |cV358ssn7a0f7jZOw...|            11|              20110915|
|  0|   1|      |9bzDeJP6sQodK73K5...|            11|              20110915|
| 32|   6|female|WFLY3s7z4EZsieHCt...|             9|              2011091

pyspark.rdd.PipelinedRDD

# Establishing Spark DataFrame Directly

In [33]:
member_df = sqlContext.read.format("csv").option("header","true").option("delimiter",',').load(path)
print(member_df.count())
print(member_df.printSchema())
print(member_df.show(5))

6769473
root
 |-- msno: string (nullable = true)
 |-- city: string (nullable = true)
 |-- bd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: string (nullable = true)
 |-- registration_init_time: string (nullable = true)

None
+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
+--------------------+----+---+------+--------------+----------------------+
only showing top 5 rows

None


# Establishing SparkSQL from Spark DataFrame

In [28]:
member_df.registerTempTable("member_table")
sqlContext.sql("SELECT count(*) counts FROM member_table").show()

+-------+
| counts|
+-------+
|6769473|
+-------+



# We rightnow have three kinds of data structures
- ### rdd: member_rdd 
- ### dataframe: member_df
- ### sparksql: member_sql

In [36]:
member_df.describe().show()

+-------+------------------+-----------------+-------+--------------------+------------------+----------------------+
|summary|                bd|             city| gender|                msno|    registered_via|registration_init_time|
+-------+------------------+-----------------+-------+--------------------+------------------+----------------------+
|  count|           6769473|          6769473|6769473|             6769473|           6769473|               6769473|
|   mean| 9.795794295951842|3.847357689439045|   null|                null| 5.253068739619761|  2.0145175906336136E7|
| stddev|17.925899719010886|5.478359063260501|   null|                null|2.3613983148064928|     23186.00761215203|
|    min|             -7168|                1|       |+++4vcS9aMH7KWdfh...|                -1|              20040326|
|    max|              2016|               22|   male|zzzyOgMk9MljCerbC...|                19|              20170429|
+-------+------------------+-----------------+-------+--